# Links

In [27]:
import pandas as pd
import os
from utils.youtube_extractor import extrair_links_playlist  # Ajuste conforme o caminho do seu módulo

# URL da playlist do YouTube
url_playlist = "https://www.youtube.com/playlist?list=PLcwPvhFHJ_F83aRklVj8vc6f4zIZafMKn"

# Obter nome da playlist e informações dos vídeos
nome_playlist, videos_info = extrair_links_playlist(url_playlist)

# Converter a lista de dicionários em DataFrame
videos_df = pd.DataFrame(videos_info)

# Exibindo os resultados
print("Nome da Playlist:", nome_playlist)
videos_df.head()

100%|██████████| 20/20 [00:26<00:00,  1.31s/it]

Nome da Playlist: Itaú Views | 5ª Temporada


,titulo,url,duração,visualizações,avaliação,autor,descrição,data_publicação,curtidas,keywords,número_curtidas,número_descurtidas
0,Privatização da Sabesp: o que esperar pela fre...,https://youtube.com/watch?v=nlpEgrjDC-8,1941,356,None,Itaú Views | 5ª Temporada,None,2023-12-18,None,[],None,None
1,Qual o melhor trade com renda fixa?,https://youtube.com/watch?v=8nxRcMYud8g,1769,621,None,Itaú Views | 5ª Temporada,None,2023-12-11,None,[],None,None
2,Um raio-X da China,https://youtube.com/watch?v=Wp5mBfznTyk,1299,402,None,Itaú Views | 5ª Temporada,None,2023-12-04,None,[],None,None
3,AI e a corrida da tecnologia,https://youtube.com/watch?v=Kr9z9GofAZw,2040,246,None,Itaú Views | 5ª Temporada,None,2023-11-27,None,[],None,None
4,Ibovespa em 145 mil pontos,https://youtube.com/watch?v=bq5Q_Fjd8Ms,1653,617,None,Itaú Views | 5ª Temporada,None,2023-11-20,None,[],None,None


In [33]:
# Salvar links
videos_df.to_csv(os.path.join("../data/links/",f"links_{nome_playlist}.csv"), index=False)

# Video

In [2]:
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import (
    OpenAIWhisperParser,
)
from langchain.document_loaders.parsers.audio import OpenAIWhisperParserLocal

In [3]:
# set a flag to switch between local and remote parsing
# change this to True if you want to use local parsing
local = True

In [4]:
import pandas as pd
videos_df = pd.read_csv("../data/links/links_Itaú Views | 5ª Temporada.csv")
url_menor_video = videos_df[videos_df['duração'] == videos_df['duração'].min()].url.values[0]

In [5]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
processor.get_decoder_prompt_ids(language="portuguese", task="transcribe")

/home/vicosbe/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[(1, 50267), (2, 50359), (3, 50363)]

In [6]:
# Two Karpathy lecture videos
urls = videos_df.url.values.tolist() # ['https://youtube.com/watch?v=plW9ZiMFcOM']  #[url_menor_video]

# Directory to save audio files
save_dir = "../data/videos/"


# Transcribe the videos to text
if local:
    loader = GenericLoader(
        # exemplo "../data/videos/Balanço dos Balanços 2T23.m4a!" necessario passar a lingua por conta do erro: Multiple languages detected when trying to predict the most likely target language for transcription. It is currently not supported to transcribe to different languages in a single batch. Please make sure to either force a single language by passing `language='...'` or make sure all input audio is of the same language.
        YoutubeAudioLoader(urls, save_dir), 
        
        OpenAIWhisperParserLocal(chunk_length=20, # quando chunk=30, houve alguns bugs, palavras repetidas na transcrição
                                                                     forced_decoder_ids=processor.get_decoder_prompt_ids(language="portuguese", task="transcribe")
                                                                     )
        
    )
else:
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

Using the following model:  openai/whisper-small


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[youtube] Extracting URL: https://youtube.com/watch?v=plW9ZiMFcOM
[youtube] plW9ZiMFcOM: Downloading webpage
[youtube] plW9ZiMFcOM: Downloading ios player API JSON
[youtube] plW9ZiMFcOM: Downloading android player API JSON


[youtube] plW9ZiMFcOM: Downloading m3u8 information
[info] plW9ZiMFcOM: Downloading 1 format(s): 140
[download] ../data/videos//Balanço dos Balanços 2T23.m4a has already been downloaded
[download] 100% of   28.54MiB
[ExtractAudio] Not converting audio ../data/videos//Balanço dos Balanços 2T23.m4a; file is already in target format m4a
Transcribing part ../data/videos/AI e a corrida da tecnologia.m4a!
Transcribing part ../data/videos/Balanço dos Balanços 2T23.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Agro, Aço, Mineração, Papel & Celulose.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Fundos Imobiliários.m4a!
Transcribing part ../data/videos/Buy Side & Sell Side ｜ Óleo & Gás.m4a!
Transcribing part ../data/videos/Como investir em créditos privados.m4a!
Transcribing part ../data/videos/El Niño： Impactos e Oportunidades para os investidores.m4a!
Transcribing part ../data/videos/Especial Macro Vision： um olhar sobre a bolsa brasileira.m4a!
Transcribing 

/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcribing part ../data/videos/Tensões globais e os efeitos para o Brasil.m4a!


/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Transcribing part ../data/videos/Um raio-X da China.m4a!


/home/vicosbe/.local/lib/python3.11/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [12]:
docs[0]

Document(page_content=' Oi pessoal, aqui é Marcelo Serrâneo, esse é mais um episódio do podcast Talvius. Antes de começar, eu deixo convite pra você nos acompanhar também nas outras plataformas. A gente tem criado alguns conteúdos complementares no Instagram e a gente disponibiliza todos os relatórios aqui do research da nossa área de pesquisa macro no Telegram. Deixando mais um convite, as edições do podcast agora tão em vídeo tanto no youtube quanto no próprio Spotify onde a gente estreia hoje a edição desse podcast em vídeo bom o assunto de hoje é tecnologia a ideia a gente falar um pouco sobre as tendências de investimentos em tecnologia, falar sobre AI, que é o tema que domina um pouco esse assunto, a gente vai entender um pouco como os convidados aqui presentes têm olhado para a BigTex, quais são as top picks quando a gente olha para esses segmentos e entender se há espaço para mais valorização ainda desse setor comparativamente ao índice e ao mercado como um tudo. Então para col

In [5]:
from utils.document_manager import save_docs_to_csv, load_docs_from_csv
# Exemplo de uso das funções
csv_file_path = '../data/transcripts/transcripts.csv'

In [14]:
# Supondo que 'docs' é a lista de documentos que você deseja salvar
save_docs_to_csv(docs, csv_file_path)


# Resumos

In [2]:
from utils.document_manager import save_docs_to_csv, load_docs_from_csv
# Carregar os documentos de volta para o formato LangChain Document
csv_file_path = '../data/transcripts/transcripts.csv'
loaded_docs = load_docs_from_csv(csv_file_path)
loaded_docs[0]

Documentos carregados de ../data/transcripts/transcripts.csv


Document(page_content=' Oi pessoal, aqui é Marcelo Serrâneo, esse é mais um episódio do podcast Talvius. Antes de começar, eu deixo convite pra você nos acompanhar também nas outras plataformas. A gente tem criado alguns conteúdos complementares no Instagram e a gente disponibiliza todos os relatórios aqui do research da nossa área de pesquisa macro no Telegram. Deixando mais um convite, as edições do podcast agora tão em vídeo tanto no youtube quanto no próprio Spotify onde a gente estreia hoje a edição desse podcast em vídeo bom o assunto de hoje é tecnologia a ideia a gente falar um pouco sobre as tendências de investimentos em tecnologia, falar sobre AI, que é o tema que domina um pouco esse assunto, a gente vai entender um pouco como os convidados aqui presentes têm olhado para a BigTex, quais são as top picks quando a gente olha para esses segmentos e entender se há espaço para mais valorização ainda desse setor comparativamente ao índice e ao mercado como um tudo. Então para col

In [76]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct") # Lembre de salvar a chave da OpenAI como variável de ambiente
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [77]:
texts = [text_splitter.split_documents([i]) for i in loaded_docs]

In [78]:
texts.__len__()

20

In [79]:
texts[:3]

[[Document(page_content='Oi pessoal, aqui é Marcelo Serrâneo, esse é mais um episódio do podcast Talvius. Antes de começar, eu deixo convite pra você nos acompanhar também nas outras plataformas. A gente tem criado alguns conteúdos complementares no Instagram e a gente disponibiliza todos os relatórios aqui do research da nossa área de pesquisa macro no Telegram. Deixando mais um convite, as edições do podcast agora tão em vídeo tanto no youtube quanto no próprio Spotify onde a gente estreia hoje a edição desse podcast em vídeo bom o assunto de hoje é tecnologia a ideia a gente falar um pouco sobre as tendências de investimentos em tecnologia, falar sobre AI, que é o tema que domina um pouco esse assunto, a gente vai entender um pouco como os convidados aqui presentes têm olhado para a BigTex, quais são as top picks quando a gente olha para esses segmentos e entender se há espaço para mais valorização ainda desse setor comparativamente ao índice e ao mercado como um tudo. Então para co

In [80]:
from langchain.chains import (
    StuffDocumentsChain,
    LLMChain,
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
)
from langchain.prompts import PromptTemplate

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)
document_variable_name = "context"
# The prompt here should take as an input variable the
# `document_variable_name`
prompt = PromptTemplate.from_template(
    "Summarize this content, use brazilian portuguese: {context}"
)
llm_chain = LLMChain(llm=llm, prompt=prompt)
# We now define how to combine these summaries
reduce_prompt = PromptTemplate.from_template(
    "Combine these summaries, use brazilian portuguese: {context}"
)
reduce_llm_chain = LLMChain(llm=llm, prompt=reduce_prompt)
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name
)
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
)
chain = MapReduceDocumentsChain(
    llm_chain=llm_chain,
    reduce_documents_chain=reduce_documents_chain,
)

In [81]:
import time
from tqdm import tqdm  # Importando a biblioteca tqdm para a barra de progresso

summaries_pt = []
max_retries = 20  # Número máximo de tentativas por texto

# Supondo que 'texts' seja uma lista de textos a serem processados e 'chain' seja pré-configurada corretamente
for i_text in tqdm(texts):
    retries = 0
    while retries <= max_retries:
        if retries == max_retries:
            print(f"Erro ao processar texto: {e}")
            raise 
            
        try:
            # Executa o processamento de cada texto
            summary = chain.run(i_text)
            if summary == '':  # Verifica se o resumo está vazio
                print("Alerta: Resumo vazio recebido, tentando novamente...")
                time.sleep(30)  # Espera 4 segundos antes de tentar novamente
                retries += 1  # Incrementa o contador de tentativas
                continue  # Continua no loop enquanto para tentar novamente
            summaries_pt.append(summary)  # Adiciona o resumo à lista se não estiver vazio
            break  # Sai do loop interno se o resumo for válido
        except Exception as e:
            print(f"Erro ao processar texto: {e}")
            break  # Sai do loop em caso de erro para evitar tentativas infinitas
    time.sleep(4)  # Delay de 4 segundos após processar cada texto

# Nota: Se um texto exceder o número máximo de tentativas, ele não será adicionado à lista 'summaries_pt'


100%|██████████| 20/20 [05:17<00:00, 15.89s/it]


In [66]:
summaries_pt

['\n\nEm resumo, a tecnologia é um setor dinâmico e forte, com perspectivas positivas para o futuro, especialmente no campo da inteligência artificial. As grandes empresas de tecnologia, como Microsoft e Amazon, estão liderando o mercado, mas também há espaço para startups e colaboração entre as empresas. A infraestrutura é fundamental para o funcionamento da tecnologia, seguida pelo software e aplicações. A Microsoft se destaca com seu assistente de IA, enquanto a Amazon é a principal escolha para investimentos no próximo ano. A Nvidia também é uma empresa promissora, com seu foco em semicondutores e chips para suportar a IA. No longo prazo, Microsoft e Nvidia são excelentes empresas, mas a Amazon tem um upside mais claro para os próximos 12 meses. A cibersegurança também é um setor a ser observado, com empresas como Microsoft, Palo Alto e CrowdStrike investindo em soluções de segurança. No geral, a tecnologia continua sendo uma corrida de inovação e eficiência, com muitas oportunidad

In [97]:
import pandas as pd
import os

# Criar um DataFrame usando pandas
df = pd.DataFrame({
    'Summary': summaries_pt,
    'Source': [i.metadata['source'] for i in loaded_docs]
})

# Definir o caminho do diretório onde o arquivo CSV será salvo
directory_path = '../data/summaries/'
csv_file_name = 'summaries.csv'
csv_file_path = os.path.join(directory_path, csv_file_name)

# Criar o diretório se ele não existir
os.makedirs(directory_path, exist_ok=True)

# Salvar o DataFrame como CSV
df.to_csv(csv_file_path, index=False)

print(f"DataFrame salvo com sucesso em {csv_file_path}")


DataFrame salvo com sucesso em ../data/summaries/summaries.csv


In [100]:
df.head().iloc[0].values

array(['\n\nNesta reunião, discutimos sobre as mudanças na OpenAI e a importância da segurança na inovação tecnológica. Também falamos sobre a estratégia de lançamento de produtos no mercado e como a Microsoft e outras empresas estão se posicionando nesse cenário. No próximo ano, veremos um aumento no uso de inteligência artificial em aplicações e um crescimento no mercado de segurança cibernética. A Amazon é vista como a principal empresa do setor tecnológico, com potencial de crescimento nos próximos anos, seguida pela Meta e Microsoft. Agradecemos a participação de Thiago e Guilherme nesta discussão. Com certeza, o campo da tecnologia é muito promissor e ainda veremos muitas mudanças e avanços nessa área.',
       '../data/videos/AI e a corrida da tecnologia.m4a'], dtype=object)

In [39]:
import time
from tqdm import tqdm  # Importando a biblioteca tqdm para a barra de progresso
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
summaries = []
# Supondo que 'texts' seja uma lista de textos a serem processados e 'chain' seja pré-configurada corretamente
for i_text in tqdm(texts):
    try:
        # Executa o processamento de cada texto
        summaries.append(chain.run(i_text))  # Certifique-se de que chain.run() é chamado corretamente com o texto
        time.sleep(4)  # Delay de 4 segundos após processar cada texto
    except Exception as e:
        print(f"Erro ao processar texto: {e}")



 75%|███████▌  | 15/20 [01:59<00:30,  6.05s/it]

Erro ao processar texto: Sorry! We've encountered an issue with repetitive patterns in your prompt. Please try again with a different prompt.


100%|██████████| 20/20 [02:42<00:00,  8.14s/it]


In [40]:
summaries

['\n\nThe podcast Talvius discusses the dominance of AI in the tech industry and its potential for future growth. Analysts from Quinea Investimentos and Itaú BBA discuss the impact of macroeconomic factors on the sector, with a focus on companies like Microsoft, Amazon, and Meta. The recent change in leadership at OpenAI is also mentioned, along with the potential for growth in AI infrastructure and applications. The speaker believes that Amazon, Microsoft, and Nvidia are top picks in the industry and there is a clear upside for Amazon in the next 12 months. Overall, the discussion highlights the potential for growth and success in the AI industry.',
 '\n\nMarcelo Serrano hosts the Talvius podcast, discussing the recent earnings season and its impact on the stock market. Guests Matheus Marques and Matheus Rafael highlight the importance of macro and micro factors and discuss the mixed results in various sectors. The article focuses on the performance of Brazilian bank Bradesco and its 

In [ ]:
# chat https://python.langchain.com/docs/integrations/document_loaders/youtube_audio/